In [1]:
import mysql.connector

#### 1. Create Connection

In [3]:
mydb = mysql.connector.connect(
    host='192.168.35.159', 
    port=3306,
    user='newrandom', 
    password=input()
    )

#### Create Connection 예제 1
- Local Database 연결

In [5]:
local = mysql.connector.connect(
    host='localhost',
    user = 'root',
    password=input()

)

#### Create Connection 예제2
- AWS RDS(database -1) 연결

In [6]:
remote = mysql.connector.connect(
    host = 'END POINT',
    port = 3306,
    user = 'root',
    password = 'password'       # AWS 접속 시 해당 문구들 변경할 것.
)

InterfaceError: 2003: Can't connect to MySQL server on '%-.100s:%u' (%s) (Warning: %u format: a real number is required, not str)

#### Close Database


In [7]:
local.close()

In [9]:
mydb.close()

#### Create Connection 예제 1
- Local MySQL 의 zerobase 에 연결

In [10]:
local = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = input(),
    database = 'zerobase'       # zerobase database가 있을 때 가능
)

ProgrammingError: 1049 (42000): Unknown database 'zerobase'

In [11]:
local.close()

- AWS RDS(database-1)의 zerobase 에 연결

In [12]:
remote = mysql.connector.connect(
    host = 'ENDPOINT',
    port=  3306,
    user = 'admin', 
    password = 'zerobase',
    database = 'zerobase'
)

InterfaceError: 2003: Can't connect to MySQL server on '%-.100s:%u' (%s) (Warning: %u format: a real number is required, not str)

In [13]:
remote.close()

NameError: name 'remote' is not defined

# Execute SQL
쿼리를 실행하기 위한 코드

In [1]:
import mysql.connector

#### Execute SQL 예제1
- table 생성

In [4]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
cur.execute('CREATE TABLE sql_file (ID int, filename varchar(16))')

remote.close()

#### Execute SQL 예제2
- 테이블 삭제

In [5]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
cur.execute('DROP TABLE sql_file')

remote.close()

#### Execute SQL File 예제 1
- test03.sql파일 생성

- test03.sql 파일 실행

In [2]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
sql = open('./test03.sql').read()      # 실행할 sql 파일의 경로
cur.execute(sql)

remote.close()

#### Execute SQL File 예제 2
- SQL file 내에 Query가 여러개 존재하는 경우

- test04.sql 생성

- 실행

In [11]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
sql = open('./test04.sql').read()
# cur.execute(sql)      ## 다중쿼리는 일반적인 execute 로 할 수 없음


## 다중 쿼리인 경우에는 multi=True 옵션을 넣어주어야 한다.
for result_iterator in cur.execute(sql, multi=True):
    if result_iterator.with_rows:               ## boolean 형태, row면 True, row가 아니면 False
        print(result_iterator.fetchall())       ## 실행된 결과를 한줄씩 보여줌.
    else:
        print(result_iterator.statement)        ## statement ?

remote.commit()
remote.close()

## sql > python > pandas


In [41]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)
cur.execute('select * from sql_file')
result = cur.fetchall()

remote.close()

In [44]:
import pandas as pd
df = pd.DataFrame(result)
df

,0,1
0,1,test01.sql
1,2,test02.sql
2,3,test03.sql
3,4,test04.sql


# Read CSV

- police_station.csv 파일 pandas로 읽어와 데이터 확인


In [45]:
import pandas as pd
df = pd.read_csv('../csv/police_station.csv', )
df.head()

,서울특별시경찰청,서울시 종로구 사직로8길 31
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


#### CSV to SQL

In [54]:
import mysql.connector
import pandas as pd

remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=input(),
    database='zerobase'
)

cur = remote.cursor(buffered=True)

# ===============DataFrame load================== #
df = pd.read_csv('../csv/police_station.csv', header=None)

query = 'INSERT INTO police_station (name, address) VALUES (%s,%s)'

for i , row in df.iterrows():
    cur.execute(query, tuple(row))
    print(tuple(row))
    remote.commit()

# cur.execute('SELECT * FROM police_station')

# result = cur.fetchall()
# for row in result:
#     print(row)


remote.close()

('서울특별시경찰청', '서울시 종로구 사직로8길 31')
('서울중부경찰서', '서울특별시 중구 수표로 27')
('서울종로경찰서', '서울특별시 종로구 율곡로 46')
('서울남대문경찰서', '서울특별시 중구 한강대로 410')
('서울서대문경찰서', '서울특별시 서대문구 통일로 113')
('서울혜화경찰서', '서울특별시 종로구 창경궁로 112-16')
('서울용산경찰서', '서울특별시 용산구 원효로89길 24')
('서울성북경찰서', '서울특별시 성북구 보문로 170')
('서울동대문경찰서', '서울특별시 동대문구 약령시로 21길 29')
('서울마포경찰서', '서울특별시 마포구 마포대로 183')
('서울영등포경찰서', '서울특별시 영등포구 국회대로 608')
('서울성동경찰서', '서울특별시 성동구 왕십리광장로 9')
('서울동작경찰서', '서울특별시 동작구 노량진로 148')
('서울광진경찰서', '서울특별시 광진구 광나루로 447 광진소방서 임시청사 (능동)')
('서울서부경찰서', '서울특별시 은평구 은평로9길 15')
('서울강북경찰서', '서울특별시 강북구 오패산로 406')
('서울금천경찰서', '서울특별시 관악구 남부순환로 1435')
('서울중랑경찰서', '서울특별시 중랑구 중랑역로 137')
('서울강남경찰서', '서울특별시 강남구 테헤란로 114길 11')
('서울관악경찰서', '서울특별시 관악구 관악로5길 33')
('서울강서경찰서', '서울특별시 양천구 화곡로 73')
('서울강동경찰서', '서울특별시 강동구 성내로 33')
('서울종암경찰서', '서울특별시 성북구 종암로 135')
('서울구로경찰서', '서울특별시 구로구 가마산로 235')
('서울서초경찰서', '서울특별시 서초구 반포대로 179')
('서울양천경찰서', '서울특별시 양천구 목동동로 99')
('서울송파경찰서', '서울특별시 송파구 중대로 221')
('서울노원경찰서', '서울특별시 노원구 노원로 283')
('서울방배경찰서', '서울특별시 서초구 방배천로 5

In [58]:
import mysql.connector
import pandas as pd

remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=input(),
    database='zerobase'
)

cur = remote.cursor(buffered=True)

df = pd.read_csv('../csv/2020_crime.csv', encoding='euc-kr')

query = 'INSERT INTO crime_status (year, police_station, crime_type, status_type, case_number) VALUES ("2020", %s, %s, %s, %s)'

for i, row in df.iterrows():
    cur.execute(query, tuple(row))
    print(tuple(row))
    remote.commit()

print('============================')
cur.execute('SELECT * FROM crime_status')

result = cur.fetchall()
for row in result:
    print(row)

print('======================')
df_ = pd.DataFrame(result)
df_.head()

remote.close()

('중부', '살인', '발생', 1)
('중부', '살인', '검거', 1)
('중부', '강도', '발생', 3)
('중부', '강도', '검거', 4)
('중부', '강간,추행', '발생', 113)
('중부', '강간,추행', '검거', 76)
('중부', '절도', '발생', 943)
('중부', '절도', '검거', 495)
('중부', '폭력', '발생', 997)
('중부', '폭력', '검거', 830)
('종로', '살인', '발생', 9)
('종로', '살인', '검거', 6)
('종로', '강도', '발생', 4)
('종로', '강도', '검거', 4)
('종로', '강간,추행', '발생', 115)
('종로', '강간,추행', '검거', 93)
('종로', '절도', '발생', 602)
('종로', '절도', '검거', 284)
('종로', '폭력', '발생', 964)
('종로', '폭력', '검거', 792)
('남대문', '살인', '발생', 1)
('남대문', '살인', '검거', 1)
('남대문', '강도', '발생', 2)
('남대문', '강도', '검거', 2)
('남대문', '강간,추행', '발생', 73)
('남대문', '강간,추행', '검거', 52)
('남대문', '절도', '발생', 699)
('남대문', '절도', '검거', 368)
('남대문', '폭력', '발생', 579)
('남대문', '폭력', '검거', 494)
('서대문', '살인', '발생', 3)
('서대문', '살인', '검거', 3)
('서대문', '강도', '발생', 2)
('서대문', '강도', '검거', 1)
('서대문', '강간,추행', '발생', 159)
('서대문', '강간,추행', '검거', 149)
('서대문', '절도', '발생', 1077)
('서대문', '절도', '검거', 524)
('서대문', '폭력', '발생', 1292)
('서대문', '폭력', '검거', 1029)
('혜화', '살인', '발생', 0)
('혜화', 

,0,1,2,3,4
0,2020,중부,살인,발생,1
1,2020,중부,살인,검거,1
2,2020,중부,강도,발생,3
3,2020,중부,강도,검거,4
4,2020,중부,"강간,추행",발생,113


## CCTV table 만들기

In [60]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)
# query = 'CREATE TABLE CCTV (GU CHAR, TOTAL INT, 2013 INT, 2014 INT, 2015 INT, 2016 INT)'
query = 'CREATE TABLE cctv (기관명 varchar(8), 소계 int, 2013년도_이전 int, 2014년 int, 2015년 int, 2016년 int)'
cur.execute(query)
remote.commit()
remote.close()

### CSV에서 Pandas로 불러오기

In [ ]:
import mysql.connector
import pandas as pd

df = pd.read_csv('../csv/01. Seoul_CCTV.csv')
df.head()

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


### data를 CCTV SQL에 넣기

In [61]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)
query = 'INSERT INTO CCTV VALUES (%s, %s, %s, %s, %s, %s)'

for i, row in df.iterrows():
    cur.execute(query, tuple(row))
    print(tuple(row))
    remote.commit()

('강남구', 3238, 1292, 430, 584, 932)
('강동구', 1010, 379, 99, 155, 377)
('강북구', 831, 369, 120, 138, 204)
('강서구', 911, 388, 258, 184, 81)
('관악구', 2109, 846, 260, 390, 613)
('광진구', 878, 573, 78, 53, 174)
('구로구', 1884, 1142, 173, 246, 323)
('금천구', 1348, 674, 51, 269, 354)
('노원구', 1566, 542, 57, 451, 516)
('도봉구', 825, 238, 159, 42, 386)
('동대문구', 1870, 1070, 23, 198, 579)
('동작구', 1302, 544, 341, 103, 314)
('마포구', 980, 314, 118, 169, 379)
('서대문구', 1254, 844, 50, 68, 292)
('서초구', 2297, 1406, 157, 336, 398)
('성동구', 1327, 730, 91, 241, 265)
('성북구', 1651, 1009, 78, 360, 204)
('송파구', 1081, 529, 21, 68, 463)
('양천구', 2482, 1843, 142, 30, 467)
('영등포구', 1277, 495, 214, 195, 373)
('용산구', 2096, 1368, 218, 112, 398)
('은평구', 2108, 1138, 224, 278, 468)
('종로구', 1619, 464, 314, 211, 630)
('중구', 1023, 413, 190, 72, 348)
('중랑구', 916, 509, 121, 177, 109)


#### cctv 테이블의 데이터 조회를 하여 확인하기

In [65]:
query = 'SELECT * FROM CCTV'
cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

remote.close()

('강남구', 3238, 1292, 430, 584, 932)
('강동구', 1010, 379, 99, 155, 377)
('강북구', 831, 369, 120, 138, 204)
('강서구', 911, 388, 258, 184, 81)
('관악구', 2109, 846, 260, 390, 613)
('광진구', 878, 573, 78, 53, 174)
('구로구', 1884, 1142, 173, 246, 323)
('금천구', 1348, 674, 51, 269, 354)
('노원구', 1566, 542, 57, 451, 516)
('도봉구', 825, 238, 159, 42, 386)
('동대문구', 1870, 1070, 23, 198, 579)
('동작구', 1302, 544, 341, 103, 314)
('마포구', 980, 314, 118, 169, 379)
('서대문구', 1254, 844, 50, 68, 292)
('서초구', 2297, 1406, 157, 336, 398)
('성동구', 1327, 730, 91, 241, 265)
('성북구', 1651, 1009, 78, 360, 204)
('송파구', 1081, 529, 21, 68, 463)
('양천구', 2482, 1843, 142, 30, 467)
('영등포구', 1277, 495, 214, 195, 373)
('용산구', 2096, 1368, 218, 112, 398)
('은평구', 2108, 1138, 224, 278, 468)
('종로구', 1619, 464, 314, 211, 630)
('중구', 1023, 413, 190, 72, 348)
('중랑구', 916, 509, 121, 177, 109)


#### 조회된 데이터를 Pandas로 변환하여 출력합니다.

In [64]:
df_ = pd.DataFrame(result)
df_.head()

,0,1,2,3,4,5
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


### Primary key and Foreign key Question

#### 1. study_id 가 primary key, patient_id 가 person 테이블의 pid와 연결된 foreign key로 지정된 study 테이블 만들기

In [1]:
import mysql.connector

In [10]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port=3306,
    user = 'newrandom',
    password=input(),
    database = 'zerobase'
)
cur = remote.cursor(buffered=True)

# ===========#
query = 'CREATE TABLE study (study_id int not null, study_date date, study_time time, patient_id int, \
        CONSTRAINT PK_study PRIMARY KEY (study_id),\
        CONSTRAINT FK_person FOREIGN KEY (patient_id) REFERENCES person(pid))'

cur.execute(query)
remote.commit()

remote.close()

#### 2. 생성한 테이블의 PRIMARY KEY 를 삭제하세요

In [12]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port=3306,
    user = 'newrandom',
    password=input(),
    database = 'zerobase'
)
cur = remote.cursor(buffered=True)

# ===========#
query = 'ALTER TABLE study DROP PRIMARY KEY'

cur.execute(query)
remote.commit()

remote.close()

#### 3. 생성한 테이블의 FOREIGN KEY를 삭제하세요

In [14]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port=3306,
    user = 'newrandom',
    password=input(),
    database = 'zerobase'
)
cur = remote.cursor(buffered=True)

# ===========#
query = 'ALTER TABLE study DROP FOREIGN KEY FK_person'

cur.execute(query)
remote.commit()

remote.close()

ProgrammingError: 1091 (42000): Can't DROP 'FK_person'; check that column/key exists

#### 4. study 테이블의 patient_id를 person 테이블의 pid와 연결된 FOREIGN KEY로 등록하세요

In [15]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port=3306,
    user = 'newrandom',
    password=input(),
    database = 'zerobase'
)
cur = remote.cursor(buffered=True)

# ===========#
query = 'ALTER TABLE study ADD CONSTRAINT FK_person FOREIGN KEY (patient_id) REFERENCES person(pid)'

cur.execute(query)
remote.commit()

remote.close()

#### 5. study 테이블의 study_id 를 PRIMARY KEY 로 등록하세요.

In [16]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port=3306,
    user = 'newrandom',
    password=input(),
    database = 'zerobase'
)
cur = remote.cursor(buffered=True)

# ===========#
query = 'ALTER TABLE study ADD CONSTRAINT PK_study PRIMARY KEY (study_id)'

cur.execute(query)
remote.commit()

remote.close()

# Aggregate Functions (집계함수)

### 1. Count

- crime_status 테이블에서 경찰서는 총 몇군데인가?


In [20]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port=3306,
    user = 'newrandom',
    password=input(),
    database = 'zerobase'
)
cur = remote.cursor(buffered=True)

# ===========#
query = 'SELECT COUNT(DISTINCT police_station) FROM crime_status'

cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()

[(31,)]


- crime_type 은 총 몇가지인가?

In [21]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port=3306,
    user = 'newrandom',
    password= input(),
    database = 'zerobase'
)
cur = remote.cursor(buffered=True)

# ===========#
query = 'SELECT COUNT(DISTINCT crime_type) FROM crime_status'

cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()

[(6,)]


### 2. Sum

- 범죄 총 발생 건수는

In [23]:
password = input()

In [25]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# =============== #
query = 'SELECT SUM(case_number) FROM crime_status WHERE status_type = "발생"'

cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()

[(Decimal('92679'),)]


- 살인의 총 발생 건수는

In [26]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# =============== #
query = 'SELECT SUM(case_number) FROM crime_status\
        WHERE crime_type LIKE "%살인%" AND status_type = "발생"'

cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()

[(Decimal('141'),)]


- 중부 경찰서에서 검거된 총 범죄 건수는

In [29]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# =============== #
query = 'SELECT SUM(case_number) FROM crime_status\
         WHERE police_station = "중부"\
         AND status_type = \'검거\''

cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()

[(Decimal('1406'),)]


#### police_station table에서 경찰서는 총 몇개이고, 각각 경찰서 이름은 무엇인지 확인

In [33]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# =============== #
query = 'SELECT COUNT(DISTINCT name) FROM police_station;'

cur.execute(query)
answer = cur.fetchall()
print('경찰서 개수 : ',answer)

query = 'SELECT name FROM police_station;'
cur.execute(query)
answer = cur.fetchall()
for _ in answer:
    
    print(answer.index(_), '경찰서 이름 :', _)

remote.close()

경찰서 개수 :  [(32,)]
0 경찰서 이름 : ('서울특별시경찰청',)
1 경찰서 이름 : ('서울중부경찰서',)
2 경찰서 이름 : ('서울종로경찰서',)
3 경찰서 이름 : ('서울남대문경찰서',)
4 경찰서 이름 : ('서울서대문경찰서',)
5 경찰서 이름 : ('서울혜화경찰서',)
6 경찰서 이름 : ('서울용산경찰서',)
7 경찰서 이름 : ('서울성북경찰서',)
8 경찰서 이름 : ('서울동대문경찰서',)
9 경찰서 이름 : ('서울마포경찰서',)
10 경찰서 이름 : ('서울영등포경찰서',)
11 경찰서 이름 : ('서울성동경찰서',)
12 경찰서 이름 : ('서울동작경찰서',)
13 경찰서 이름 : ('서울광진경찰서',)
14 경찰서 이름 : ('서울서부경찰서',)
15 경찰서 이름 : ('서울강북경찰서',)
16 경찰서 이름 : ('서울금천경찰서',)
17 경찰서 이름 : ('서울중랑경찰서',)
18 경찰서 이름 : ('서울강남경찰서',)
19 경찰서 이름 : ('서울관악경찰서',)
20 경찰서 이름 : ('서울강서경찰서',)
21 경찰서 이름 : ('서울강동경찰서',)
22 경찰서 이름 : ('서울종암경찰서',)
23 경찰서 이름 : ('서울구로경찰서',)
24 경찰서 이름 : ('서울서초경찰서',)
25 경찰서 이름 : ('서울양천경찰서',)
26 경찰서 이름 : ('서울송파경찰서',)
27 경찰서 이름 : ('서울노원경찰서',)
28 경찰서 이름 : ('서울방배경찰서',)
29 경찰서 이름 : ('서울도봉경찰서',)
30 경찰서 이름 : ('서울수서경찰서',)
31 경찰서 이름 : ('서울은평경찰서',)


#### crime_status 에서 status_type은 총 몇개이고, 각각 타입은 무엇인지 확인하세요.

In [34]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

query = 'SELECT COUNT(DISTINCT status_type) FROM crime_status'

cur.execute(query)
answer = cur.fetchall()
print(answer)

# ================= #
print( '=========')
query = 'SELECT DISTINCT status_type FROM crime_status'
cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()

[(2,)]
[('발생',), ('검거',)]


#### 종로경찰서와 남대문경찰서의 강도 발생 건수의 합을 구하세요.

In [35]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

query = 'SELECT SUM(case_number) FROM crime_status \
        WHERE police_station IN ("종로", "남대문")\
        AND crime_type = "강도"\
        AND status_type = "발생"'

cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()




[(Decimal('6'),)]


#### 폭력 범죄의 검거 건수의 합을 구하세요.

In [37]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

query = 'SELECT SUM(case_number) FROM crime_status \
        WHERE crime_type = "폭력" \
        AND status_type = "검거" '

cur.execute(query)
answer = cur.fetchall()
print(answer)

remote.close()


[(Decimal('41019'),)]


### 3. Avg

In [38]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# 평균 폭력 검거 건수
query = "SELECT AVG(case_number) FROM crime_status\
        WHERE crime_type LIKE '폭력' \
        AND status_type='검거'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(Decimal('1323.1935'),)]


In [39]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# 중부 경찰서 범죄 평균 발생 건수
query = "SELECT AVG(CASE_NUMBER) FROM CRIME_STATUS\
         WHERE POLICE_STATION = '중부' AND STATUS_TYPE = '발생'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(Decimal('411.4000'),)]


### 4. Min

In [40]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# 강도 발생 건수가 가장 적은 경우 몇 건
query = "SELECT MIN(CASE_NUMBER)\
        FROM CRIME_STATUS\
        WHERE CRIME_TYPE = '강도' AND STATUS_TYPE = '발생'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(1,)]


In [41]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# 중부경찰서에서 가장 낮은 검거 건수는?
query = "SELECT MIN(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE POLICE_STATION = '중부' AND STATUS_TYPE = '검거'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(1,)]


### 5. Max

In [42]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# 살인이 가장 많이 검거된 건수는
query = "SELECT MAX(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE CRIME_TYPE = '살인' AND STATUS_TYPE='검거'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(12,)]


In [43]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

# 강남경찰서에서 가장 많이 발생한 범죄 건수는?
query = "SELECT MAX(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE POLICE_STATION = '강남' AND STATUS_TYPE='발생'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(2283,)]


### 예제 1.

#### 살인의 평균 발생 건수를 검색하고 확인하세요.

In [45]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


query = "SELECT AVG(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE CRIME_TYPE='살인' AND STATUS_TYPE = '발생'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(Decimal('4.5484'),)]


#### 서초경찰서의 범죄 별 평균 검거 건수를 검색하고 확인하세요.

In [62]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


# query = "SELECT DISTINCT CRIME_TYPE, AVG(CASE_NUMBER) FROM CRIME_STATUS\
#         WHERE POLICE_STATION = '서초' \
#         AND STATUS_TYPE = '검거'\
#         GROUP BY CRIME_TYPE"
# >> [('강간,추행', Decimal('253.0000')), ('강도', Decimal('3.0000')), ('살인', Decimal('4.0000')), ('절도', Decimal('801.0000')), ('폭력', Decimal('1415.0000'))]

query = "SELECT CRIME_TYPE, AVG(CASE_NUMBER) FROM CRIME_STATUS\
    WHERE POLICE_STATION = '서초'\
    AND STATUS_TYPE = '검거'\
        GROUP BY CRIME_TYPE"
# >> [('강간,추행', Decimal('253.0000')), ('강도', Decimal('3.0000')), ('살인', Decimal('4.0000')), ('절도', Decimal('801.0000')), ('폭력', Decimal('1415.0000'))]

cur.execute(query)
print(cur.fetchall())

remote.close()

[('강간,추행', Decimal('253.0000')), ('강도', Decimal('3.0000')), ('살인', Decimal('4.0000')), ('절도', Decimal('801.0000')), ('폭력', Decimal('1415.0000'))]


#### 구로경찰서와 도봉경찰서의 평균 살인 검거 건수를 검색하고 확인하세요.

In [54]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


query = "SELECT AVG(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE POLICE_STATION IN ('구로','도봉')\
        AND CRIME_TYPE = '살인'\
        AND STATUS_TYPE = '검거'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(Decimal('3.0000'),)]


#### 광진경찰서에서 가장 낮은 범죄 검거 건수를 검색하고 확인하세요.ㅡ

In [55]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


query = "SELECT MIN(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE POLICE_STATION = '광진'\
        AND STATUS_TYPE='검거'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(2,)]


#### 성북 경찰서에서 가장 낮은 범죄 발생 건수를 검색하고 확인하세요.

In [56]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


query = "SELECT MIN(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE POLICE_STATION = '성북'\
        AND STATUS_TYPE = '발생'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(0,)]


#### 영등포경찰서의 가장 높은 범죄 발생 건수를 검색하고 확인하세요.

In [57]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


query = "SELECT MAX(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE POLICE_STATION = '영등포'\
        AND STATUS_TYPE = '발생'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(2701,)]


#### 절도 검거가 가장 많은 건수를 검색하고 확인하세요.

In [59]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


query = "SELECT MAX(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE CRIME_TYPE = '절도'\
        AND STATUS_TYPE = '검거'"

cur.execute(query)
print(cur.fetchall())

remote.close()

[(1137,)]


### 6. GROUP BY

In [60]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)


query = "SELECT POLICE_STATION FROM CRIME_STATUS\
        GROUP BY POLICE_STATION\
        ORDER BY POLICE_STATION"

cur.execute(query)
print(cur.fetchall())

remote.close()

[('강남',), ('강동',), ('강북',), ('강서',), ('관악',), ('광진',), ('구로',), ('금천',), ('남대문',), ('노원',), ('도봉',), ('동대문',), ('동작',), ('마포',), ('방배',), ('서대문',), ('서부',), ('서초',), ('성동',), ('성북',), ('송파',), ('수서',), ('양천',), ('영등포',), ('용산',), ('은평',), ('종로',), ('종암',), ('중랑',), ('중부',), ('혜화',)]


In [61]:
remote = mysql.connector.connect(
    host= '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=password,
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)

## DISTINCT 는 ORDER BY 를 사용할 수 없다.
query = "SELECT DISTINCT POLICE_STATION FROM CRIME_STATUS LIMIT 5"

cur.execute(query)
print(cur.fetchall())

remote.close()

[('중부',), ('종로',), ('남대문',), ('서대문',), ('혜화',)]


#### 경찰서 별로 총 발생 범죄 건수를 검색


In [63]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database='zerobase'
)

cur = remote.cursor(buffered=True)
query = 'SELECT POLICE_STATION, SUM(CASE_NUMBER)\
        FROM CRIME_STATUS\
            WHERE STATUS_TYPE=\'발생\'\
                GROUP BY POLICE_STATION'

cur.execute(query)
print(cur.fetchall())

remote.close()


[('강남', Decimal('4754')), ('강동', Decimal('3788')), ('강북', Decimal('2770')), ('강서', Decimal('4415')), ('관악', Decimal('5261')), ('광진', Decimal('3601')), ('구로', Decimal('4175')), ('금천', Decimal('2598')), ('남대문', Decimal('1354')), ('노원', Decimal('3743')), ('도봉', Decimal('2179')), ('동대문', Decimal('3401')), ('동작', Decimal('3200')), ('마포', Decimal('3688')), ('방배', Decimal('836')), ('서대문', Decimal('2533')), ('서부', Decimal('1362')), ('서초', Decimal('3765')), ('성동', Decimal('2362')), ('성북', Decimal('1208')), ('송파', Decimal('5410')), ('수서', Decimal('2602')), ('양천', Decimal('3216')), ('영등포', Decimal('5217')), ('용산', Decimal('2969')), ('은평', Decimal('2028')), ('종로', Decimal('1694')), ('종암', Decimal('1359')), ('중랑', Decimal('3726')), ('중부', Decimal('2057')), ('혜화', Decimal('1408'))]


#### 경찰서 별로 평균 범죄 검거 건수를 검색


In [64]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database='zerobase'
)

cur = remote.cursor(buffered=True)
query = 'SELECT POLICE_STATION, AVG(CASE_NUMBER) AS "평균건수" FROM CRIME_STATUS\
    WHERE STATUS_TYPE = "검거"\
        GROUP BY POLICE_STATION'

cur.execute(query)
print(cur.fetchall())

remote.close()


[('강남', Decimal('674.6000')), ('강동', Decimal('532.4000')), ('강북', Decimal('451.8000')), ('강서', Decimal('678.6000')), ('관악', Decimal('771.6000')), ('광진', Decimal('502.2000')), ('구로', Decimal('596.8000')), ('금천', Decimal('393.2000')), ('남대문', Decimal('183.4000')), ('노원', Decimal('516.2000')), ('도봉', Decimal('298.0000')), ('동대문', Decimal('493.0000')), ('동작', Decimal('417.2000')), ('마포', Decimal('506.8000')), ('방배', Decimal('115.2000')), ('서대문', Decimal('341.2000')), ('서부', Decimal('213.0000')), ('서초', Decimal('495.2000')), ('성동', Decimal('354.8000')), ('성북', Decimal('184.4000')), ('송파', Decimal('708.8000')), ('수서', Decimal('374.4000')), ('양천', Decimal('417.2000')), ('영등포', Decimal('674.2000')), ('용산', Decimal('411.2000')), ('은평', Decimal('293.2000')), ('종로', Decimal('235.8000')), ('종암', Decimal('205.0000')), ('중랑', Decimal('556.2000')), ('중부', Decimal('281.2000')), ('혜화', Decimal('198.8000'))]


#### 경찰서 별 평균 범죄 발생건수와 평균 범죄 검거 건수를 검색

In [67]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database='zerobase'
)

cur = remote.cursor(buffered=True)
# query = 'SELECT POLICE_STATION, STATUS_TYPE, AVG(CASE_NUMBER) FROM CRIME_STATUS\
#     GROUP BY POLICE_STATION'        ## 발생 한 경우만 조회됨
query = 'SELECT POLICE_STATION, STATUS_TYPE, AVG(CASE_NUMBER) FROM CRIME_STATUS\
    GROUP BY POLICE_STATION, STATUS_TYPE'

cur.execute(query)
print(cur.fetchall())

remote.close()


[('강남', '검거', Decimal('674.6000')), ('강남', '발생', Decimal('950.8000')), ('강동', '검거', Decimal('532.4000')), ('강동', '발생', Decimal('757.6000')), ('강북', '검거', Decimal('451.8000')), ('강북', '발생', Decimal('554.0000')), ('강서', '검거', Decimal('678.6000')), ('강서', '발생', Decimal('883.0000')), ('관악', '검거', Decimal('771.6000')), ('관악', '발생', Decimal('1052.2000')), ('광진', '검거', Decimal('502.2000')), ('광진', '발생', Decimal('720.2000')), ('구로', '검거', Decimal('596.8000')), ('구로', '발생', Decimal('835.0000')), ('금천', '검거', Decimal('393.2000')), ('금천', '발생', Decimal('519.6000')), ('남대문', '검거', Decimal('183.4000')), ('남대문', '발생', Decimal('270.8000')), ('노원', '검거', Decimal('516.2000')), ('노원', '발생', Decimal('748.6000')), ('도봉', '검거', Decimal('298.0000')), ('도봉', '발생', Decimal('435.8000')), ('동대문', '검거', Decimal('493.0000')), ('동대문', '발생', Decimal('680.2000')), ('동작', '검거', Decimal('417.2000')), ('동작', '발생', Decimal('640.0000')), ('마포', '검거', Decimal('506.8000')), ('마포', '발생', Decimal('737.6000')), ('방배', '검거', D

### 7. HAVING

#### 경찰서 별로 발생한 범죄 건수의 합이 4000 건보다 큰 경우를 검색

In [68]:
import pandas as pd

In [72]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database='zerobase'
)

cur = remote.cursor(buffered=True)
query = 'SELECT POLICE_STATION, SUM(CASE_NUMBER) FROM CRIME_STATUS\
        WHERE STATUS_TYPE = "발생"\
        GROUP BY POLICE_STATION\
            HAVING SUM(CASE_NUMBER) > 4000\
                ORDER BY SUM(CASE_NUMBER) DESC'

cur.execute(query)
answer = cur.fetchall()

df = pd.DataFrame(answer)
print(df.head(6))

remote.close()


     0     1
0   송파  5410
1   관악  5261
2  영등포  5217
3   강남  4754
4   강서  4415
5   구로  4175


#### 경찰서 별로 발생한 폭력과 절도의 범죄 건수 평균이 2000 이상인 경우를 검색

In [76]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = password,
    database='zerobase'
)

cur = remote.cursor(buffered=True)
        ## 폭력과 절도 범죄의 건수 평균이 각각 2000 이상인 경우를 검색
# query = "SELECT POLICE_STATION, CRIME_TYPE, AVG(CASE_NUMBER) '범죄건수평균' FROM CRIME_STATUS\
#     WHERE STATUS_TYPE='발생' \
#     AND CRIME_TYPE IN ('폭력', '절도')\
#     GROUP BY POLICE_STATION, CRIME_TYPE\
#     HAVING 범죄건수평균 >= 2000"

## 폭력과 절도의 범죄를 합한 평균이 2000 이상인 경우를 검색
query = "SELECT POLICE_STATION, CRIME_TYPE, AVG(CASE_NUMBER) FROM CRIME_STATUS\
    WHERE CRIME_TYPE IN ('폭력', '절도') AND STATUS_TYPE = '발생'\
        GROUP BY POLICE_STATION\
            HAVING AVG(CASE_NUMBER) >= 2000"
cur.execute(query)
answer = cur.fetchall()

df = pd.DataFrame(answer)
print(df.head(6))

remote.close()


     0   1          2
0   강남  절도  2112.0000
1   강서  절도  2067.0000
2   관악  절도  2421.5000
3   송파  절도  2552.0000
4  영등포  절도  2444.5000
